 ↓ Importing all the Pakcages needed

In [1]:
import os
import itertools
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import PIL
from PIL import ImageFile


print("All required packages are installed successfully!")


All required packages are installed successfully!


Checking if GPU is being Used for model training

In [3]:
import tensorflow as tf

print("TensorFlow is using GPU:", tf.test.is_built_with_cuda())
print("GPU Devices:", tf.config.experimental.list_physical_devices('GPU'))


TensorFlow is using GPU: True
GPU Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


 ↓ Tensorflow Version used

In [4]:
import tensorflow as tf;
print(tf.__version__)

2.10.0


Python Version Used

In [5]:
import sys
import site
print("Python Version:", sys.version)
print("Installed Libraries:", site.getsitepackages())

Python Version: 3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]
Installed Libraries: ['a:\\PJT2\\envs\\waste_sorting', 'a:\\PJT2\\envs\\waste_sorting\\lib\\site-packages']


Categorising TACO dataset as [Hazardous / Recyclable / Non-Recyclable / Organic]

In [6]:
# import os
# import shutil

# # Define source and target dataset directories
# TACO_DATASET_DIR = r"A:\PJT2\datasets\Validation\TACO"
# TARGET_DIR = r"A:\PJT2\datasets\Validation"

# CATEGORY_MAPPING = {
#     "Aerosol": "Hazardous",
#     "Aluminium blister pack": "Hazardous",
#     "Battery": "Hazardous",
#     "Cigarette": "Hazardous",

#     "Aluminium foil": "Recyclable",
#     "Clear plastic bottle": "Recyclable",
#     "Corrugated carton": "Recyclable",
#     "Drink can": "Recyclable",
#     "Drink carton": "Recyclable",
#     "Egg carton": "Recyclable",
#     "Food can": "Recyclable",
#     "Glass bottle": "Recyclable",
#     "Glass cup": "Recyclable",
#     "Glass jar": "Recyclable",
#     "Magazine paper": "Recyclable",
#     "Meal carton": "Recyclable",
#     "Metal bottle cap": "Recyclable",
#     "Metal lid": "Recyclable",
#     "Normal paper": "Recyclable",
#     "Other carton": "Recyclable",
#     "Other plastic bottle": "Recyclable",
#     "Other plastic container": "Recyclable",
#     "Paper bag": "Recyclable",
#     "Paper cup": "Recyclable",
#     "Pizza box": "Recyclable",
#     "Plastic bottle cap": "Recyclable",
#     "Plastic lid": "Recyclable",
#     "Pop tab": "Recyclable",
#     "Scrap metal": "Recyclable",
#     "Six pack rings": "Recyclable",
#     "Spread tub": "Recyclable",
#     "Toilet tube": "Recyclable",
#     "Tupperware": "Recyclable",
#     "Wrapping paper": "Recyclable",

#     "Broken glass": "Non-Recyclable",
#     "Crisp packet": "Non-Recyclable",
#     "Disposable food container": "Non-Recyclable",
#     "Disposable plastic cup": "Non-Recyclable",
#     "Foam cup": "Non-Recyclable",
#     "Foam food container": "Non-Recyclable",
#     "Garbage bag": "Non-Recyclable",
#     "Other plastic": "Non-Recyclable",
#     "Other plastic wrapper": "Non-Recyclable",
#     "Plastic film": "Non-Recyclable",
#     "Plastic gloves": "Non-Recyclable",
#     "Plastic straw": "Non-Recyclable",
#     "Plastic utensils": "Non-Recyclable",
#     "Polypropylene bag": "Non-Recyclable",
#     "Rope & strings": "Non-Recyclable",
#     "Shoe": "Non-Recyclable",
#     "Single-use carrier bag": "Non-Recyclable",
#     "Squeezable tube": "Non-Recyclable",
#     "Styrofoam piece": "Non-Recyclable",
#     "Tissues": "Non-Recyclable",
#     "Unlabeled litter": "Non-Recyclable",

#     "Food waste": "Organic"
# }

# # Move entire subcategory folders into the main category folders
# for subcategory, main_category in CATEGORY_MAPPING.items():
#     subcategory_path = os.path.join(TACO_DATASET_DIR, subcategory)
#     target_category_path = os.path.join(TARGET_DIR, main_category)
#     target_path = os.path.join(target_category_path, subcategory)

#     if os.path.exists(subcategory_path):
#         # Ensure main category folder exists
#         os.makedirs(target_category_path, exist_ok=True)

#         # Move the whole folder if it doesn't already exist in the target
#         if not os.path.exists(target_path):
#             shutil.move(subcategory_path, target_path)
#         else:
#             print(f"⚠️ Folder already exists: {target_path}. Merging contents.")
#             for img_file in os.listdir(subcategory_path):
#                 src = os.path.join(subcategory_path, img_file)
#                 dst = os.path.join(target_path, img_file)

#                 # Ensure unique filenames
#                 if os.path.exists(dst):
#                     base, ext = os.path.splitext(img_file)
#                     counter = 1
#                     while os.path.exists(os.path.join(target_path, f"{base}_{counter}{ext}")):
#                         counter += 1
#                     dst = os.path.join(target_path, f"{base}_{counter}{ext}")

#                 shutil.move(src, dst)
            
#             # Remove the empty folder after moving files
#             os.rmdir(subcategory_path)

# print("✅ Dataset Restructuring Completed!")


↓ DATA PREPROCESSING  

In [7]:
# import os
# import shutil
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm

# # Paths for datasets (TACO + 4 main categories)
# DATASET_PATH = "A:/PJT2/datasets"  # Root dataset folder
# TACO_DATASET_PATH = os.path.join(DATASET_PATH, "TACO")  # TACO dataset
# MAIN_CATEGORIES = ["Hazardous", "Non-Recyclable", "Recyclable", "Organic"]  # Four main categories

# # Destination folders for train/val split
# TRAIN_DIR = os.path.join(DATASET_PATH, "Train")
# VAL_DIR = os.path.join(DATASET_PATH, "Validation")

# # Ensure train/val directories exist
# os.makedirs(TRAIN_DIR, exist_ok=True)
# os.makedirs(VAL_DIR, exist_ok=True)

# def preprocess_main_categories():
#     """ Process the structured dataset with 4 main waste categories. """
#     for category in MAIN_CATEGORIES:
#         category_path = os.path.join(DATASET_PATH, category)
#         if not os.path.isdir(category_path):
#             continue

#         waste_types = os.listdir(category_path)
#         for waste_type in waste_types:
#             waste_path = os.path.join(category_path, waste_type)
#             if not os.path.isdir(waste_path):
#                 continue

#             images = [img for img in os.listdir(waste_path) if os.path.isfile(os.path.join(waste_path, img))]
#             print(f"📂 Processing {category}/{waste_type} - {len(images)} images")

#             if len(images) < 2:
#                 print(f"⚠️ Skipping {category}/{waste_type} (Not enough images)")
#                 continue

#             # Split images into training (80%) and validation (20%)
#             train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

#             # Create folders inside Train & Validation
#             train_sub_dir = os.path.join(TRAIN_DIR, category, waste_type)
#             val_sub_dir = os.path.join(VAL_DIR, category, waste_type)
#             os.makedirs(train_sub_dir, exist_ok=True)
#             os.makedirs(val_sub_dir, exist_ok=True)

#             # Move images
#             for img_name in tqdm(train_images, desc=f"🔹 Train - {category}/{waste_type}"):
#                 shutil.copy(os.path.join(waste_path, img_name), os.path.join(train_sub_dir, img_name))
#             for img_name in tqdm(val_images, desc=f"🔸 Validation - {category}/{waste_type}"):
#                 shutil.copy(os.path.join(waste_path, img_name), os.path.join(val_sub_dir, img_name))

# def preprocess_taco_dataset():
#     """ Process the flat 'TACO' dataset where each waste type is a folder. """
#     waste_types = os.listdir(TACO_DATASET_PATH)

#     for waste_type in waste_types:
#         waste_path = os.path.join(TACO_DATASET_PATH, waste_type)
#         if not os.path.isdir(waste_path):
#             continue

#         images = [img for img in os.listdir(waste_path) if os.path.isfile(os.path.join(waste_path, img))]
#         print(f"📂 Processing TACO/{waste_type} - {len(images)} images")

#         if len(images) < 2:
#             print(f"⚠️ Skipping {waste_type} (Not enough images)")
#             continue

#         # Split images into training (80%) and validation (20%)
#         train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

#         # Create folders inside Train & Validation
#         train_sub_dir = os.path.join(TRAIN_DIR, "TACO", waste_type)
#         val_sub_dir = os.path.join(VAL_DIR, "TACO", waste_type)
#         os.makedirs(train_sub_dir, exist_ok=True)
#         os.makedirs(val_sub_dir, exist_ok=True)

#         # Move images
#         for img_name in tqdm(train_images, desc=f"🔹 Train - TACO/{waste_type}"):
#             shutil.copy(os.path.join(waste_path, img_name), os.path.join(train_sub_dir, img_name))
#         for img_name in tqdm(val_images, desc=f"🔸 Validation - TACO/{waste_type}"):
#             shutil.copy(os.path.join(waste_path, img_name), os.path.join(val_sub_dir, img_name))

# # Run preprocessing for both datasets
# preprocess_main_categories()
# preprocess_taco_dataset()

# print("✅ Dataset cleaning & preprocessing completed!")


Removing Non Image Files (.pdf / .txt / .docx)

In [8]:
# import os
# import imghdr

# # Define dataset directories
# DATASET_DIRS = ["A:/PJT2/datasets"]

# def remove_invalid_files():
#     for dataset_path in DATASET_DIRS:
#         for root, _, files in os.walk(dataset_path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 if imghdr.what(file_path) is None:  # Check if file is an image
#                     print(f"🚨 Removing non-image file: {file_path}")
#                     os.remove(file_path)  # Delete the invalid file

# remove_invalid_files()
# print("✅ Non-image files removed successfully!")


In [9]:
# import os
# import psutil

# DATASET_DIRS = ["A:/PJT2/datasets"]

# def close_file_handles():
#     """ Tries to close file handles for locked files in the dataset directory. """
#     for proc in psutil.process_iter():
#         try:
#             for open_file in proc.open_files():
#                 file_path = open_file.path.replace("\\", "/")
#                 for dataset_path in DATASET_DIRS:
#                     if dataset_path in file_path:
#                         print(f"🔄 Closing file in use: {file_path} (Process: {proc.name()})")
#                         proc.terminate()
#         except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
#             pass

# close_file_handles()
# print("✅ All locked files should now be accessible.")


Conversion of .webp image to .jpeg

In [10]:
# from PIL import Image

# def convert_images_to_jpeg():
#     for dataset_path in DATASET_DIRS:
#         for root, _, files in os.walk(dataset_path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 if imghdr.what(file_path) not in ['jpeg', 'png']:  # Convert only if not already in JPEG/PNG
#                     try:
#                         img = Image.open(file_path)
#                         new_file_path = file_path.rsplit(".", 1)[0] + ".jpg"  # Change extension
#                         img.convert("RGB").save(new_file_path, "JPEG")  # Convert and save as JPEG
#                         os.remove(file_path)  # Remove old file
#                         print(f"🔄 Converted {file_path} → {new_file_path}")
#                     except Exception as e:
#                         print(f"⚠️ Could not convert {file_path}: {e}")

# convert_images_to_jpeg()
# print("✅ Image format conversion complete!")


Removing of .gif images

In [11]:
# def remove_gif_files():
#     for dataset_path in DATASET_DIRS:
#         for root, _, files in os.walk(dataset_path):
#             for file in files:
#                 if file.lower().endswith(".gif"):
#                     file_path = os.path.join(root, file)
#                     print(f"🗑️ Deleting GIF file: {file_path}")
#                     os.remove(file_path)

# remove_gif_files()
# print("✅ All GIF files have been deleted.")


Resising of Images to 224x224

In [12]:
# import tensorflow as tf
# import os

# # Define dataset paths
# TRAIN_DIR = "A:/PJT2/datasets/Train"
# VAL_DIR = "A:/PJT2/datasets/Validation"

# # Image properties
# IMG_SIZE = (224, 224)
# BATCH_SIZE = 32
# AUTOTUNE = tf.data.AUTOTUNE

# def preprocess_image(image, label):
#     """ Resize and normalize image """
#     image = tf.image.convert_image_dtype(image, tf.float32)  # Ensure correct dtype
#     image = tf.image.resize(image, IMG_SIZE)  # Resize to 224x224
#     return image, label

# def load_dataset(directory):
#     """ Load dataset from directory """
#     dataset = tf.keras.preprocessing.image_dataset_from_directory(
#         directory,
#         image_size=IMG_SIZE,
#         batch_size=BATCH_SIZE,
#         shuffle=True,
#         label_mode='categorical'
#     )
    
#     return dataset.map(preprocess_image, num_parallel_calls=AUTOTUNE).cache().shuffle(1000).prefetch(AUTOTUNE)

# # Load datasets
# train_dataset = load_dataset(TRAIN_DIR)
# val_dataset = load_dataset(VAL_DIR)

# # Verify dataset
# for images, labels in train_dataset.take(1):
#     print(f"✅ Image batch shape: {images.shape}")
#     print(f"✅ Label batch shape: {labels.shape}")


 ↓MODEL TRAINING 

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# ✅ Set Paths
DATASET_DIR = "A:/PJT2/datasets/Train"
BATCH_SIZE = 16
IMG_SIZE = (224, 224)
EPOCHS = 10
SUBSET_SIZE = 1.0  # Set to 1.0 for full training, reduce for debugging

# ✅ Create Data Generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode="categorical", subset="training"
)
val_generator = val_datagen.flow_from_directory(
    DATASET_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode="categorical", subset="validation"
)

# ✅ Debugging: Check Image Counts
total_train_images = len(train_generator.filenames)
total_val_images = len(val_generator.filenames)
total_images_in_folder = sum(len(files) for _, _, files in os.walk(DATASET_DIR))
print(f"🔹 Total images in dataset folder: {total_images_in_folder}")
print(f"✅ Training images used: {total_train_images}")
print(f"✅ Validation images used: {total_val_images}")

# ✅ Load Pretrained Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# ✅ Build Custom Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(train_generator.num_classes, activation="softmax", name="category_output")(x)

model = Model(inputs=base_model.input, outputs=output)

# ✅ Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# ✅ Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=max(1, int(len(train_generator) * SUBSET_SIZE)),
    validation_steps=max(1, int(len(val_generator) * SUBSET_SIZE)),
    epochs=EPOCHS
)

# ✅ Save Model with Fix
try:
    model.save("waste_classification_model.h5", save_format="h5")
    print("✅ Model saved successfully in HDF5 format!")
except Exception as e:
    print(f"⚠️ HDF5 save failed: {e}")
    print("🔄 Trying TensorFlow SavedModel format...")
    model.save("waste_classification_model")
    print("✅ Model saved in TensorFlow SavedModel format!")


Found 3054 images belonging to 4 classes.
Found 762 images belonging to 4 classes.
🔹 Total images in dataset folder: 3817
✅ Training images used: 3054
✅ Validation images used: 762
Epoch 1/10
  7/191 [>.............................] - ETA: 40s - loss: 1.7069 - accuracy: 0.2411

a:\PJT2\envs\waste_sorting\lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


 44/191 [=====>........................] - ETA: 44s - loss: 1.5161 - accuracy: 0.3366

a:\PJT2\envs\waste_sorting\lib\site-packages\PIL\TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


191/191 [==============================] - 92s 459ms/step - loss: 1.2476 - accuracy: 0.4398 - val_loss: 1.3269 - val_accuracy: 0.4186
Epoch 2/10
191/191 [==============================] - 88s 462ms/step - loss: 0.9249 - accuracy: 0.5897 - val_loss: 1.3709 - val_accuracy: 0.4462
Epoch 3/10
191/191 [==============================] - 90s 473ms/step - loss: 0.8190 - accuracy: 0.6568 - val_loss: 1.4492 - val_accuracy: 0.4331
Epoch 4/10
191/191 [==============================] - 89s 466ms/step - loss: 0.7447 - accuracy: 0.6863 - val_loss: 1.4903 - val_accuracy: 0.4528
Epoch 5/10
191/191 [==============================] - 88s 462ms/step - loss: 0.6802 - accuracy: 0.7210 - val_loss: 1.4849 - val_accuracy: 0.4475
Epoch 6/10
191/191 [==============================] - 92s 483ms/step - loss: 0.6345 - accuracy: 0.7410 - val_loss: 1.4820 - val_accuracy: 0.4593
Epoch 7/10
191/191 [==============================] - 89s 464ms/step - loss: 0.5931 - accuracy: 0.7688 - val_loss: 1.5104 - val_accuracy: 0.4

Saving the model 

In [ ]:
model.save("./Models/waste_classification_model.h5")

 ↓

In [ ]:
# import numpy as np
# from tensorflow.keras.preprocessing import image

# def predict_image(img_path, model, class_labels):
#     img = image.load_img("./test_imgs/Alkaline-batteries.jpg", target_size=(224, 224))
#     img_array = image.img_to_array(img) / 255.0  # Normalize
#     img_array = np.expand_dims(img_array, axis=0)  # Expand dims for model input
#     prediction = model.predict(img_array)
    
#     predicted_class = class_labels[np.argmax(prediction)]
#     confidence = np.max(prediction)
    
#     print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")

# # Example usage
# class_labels = list(train_generator.class_indices.keys())
# predict_image("A:/PJT2/test_imgs/000055.jpg", model, class_labels)


In [ ]:
# val_loss, val_acc = model.evaluate(val_generator)
# print(f"Validation Accuracy: {val_acc * 100:.2f}%")


 ↓

In [ ]:
# import numpy as np
# from tensorflow.keras.preprocessing import image

# img_path = "./test_imgs/000055.jpg"  # Update this with an actual image path
# img = image.load_img(img_path, target_size=(224, 224))  # Resize to model input size
# img_array = image.img_to_array(img) / 255.0  # Normalize
# img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# # Predict
# predictions = model.predict(img_array)
# predicted_class = np.argmax(predictions)

# print(f"Predicted Class: {predicted_class}")  # You may need to map this to class labels


 ↓

In [ ]:
# import matplotlib.pyplot as plt

# # Select some validation images
# batch = next(iter(val_generator))
# images, labels = batch

# # Make predictions
# preds = model.predict(images)
# predicted_classes = np.argmax(preds, axis=1)

# # Plot images with predicted labels
# fig, axes = plt.subplots(3, 3, figsize=(10, 10))
# axes = axes.flatten()

# for img, label, pred, ax in zip(images, labels, predicted_classes, axes):
#     ax.imshow(img)
#     ax.axis("off")
#     ax.set_title(f"True: {np.argmax(label)}, Pred: {pred}")

# plt.show()
